# Run the example

In [ ]:

# compile and run the design, nothing very interesting here
import os
from subprocess import Popen, PIPE, CalledProcessError

os.environ['VCS_HOME'] = 'path-to-vcs'
os.environ['VERDI_HOME'] = 'path-to-verdi'

with Popen('simple_example/run', stdout=PIPE, bufsize=1, universal_newlines=True) as p:
    if p.returncode != 0:
        raise CalledProcessError(p.returncode, p.args)

#  Get NPI license

In [3]:
# imports and initialization (i.e. license check)
from pynpi import npisys

npisys.init(["anything"])

1

# Load design database

We use this to extract a list of file/line locations containing non-synthesizable initial blocks

In [4]:
if not npisys.load_design(['-dbdir', 'simv.daidir']):
    print("couldn't open design")

# Find lines in "initial" blocks

We recursively go through the hierarchy looking for initial blocks, then we recursively go through initial blocks to find statements. Yes, because of the way NPI is built we recurse a lot...

In [5]:
from pynpi import lang

# find all statements in a given process (i.e. initial/always)
def find_all_statements(process, locations=[]):     
    block = process.stmt_handle()
    if (block != None):
        if block.file() != None:
            locations.append((block.file(), block.line_no()))

        statements = block.stmt_handles()
        for statement in statements:
            if statement.file() != None:
                locations.append((statement.file(), statement.line_no()))
            find_all_statements(statement, locations)
            
    statements = process.stmt_handles()
    for statement in statements:
        if sttement.file() != None:
            locations.append((statement.file(), statement.line_no()))
        find_all_statements(statement, locations)    
    

# find all initials down to a given depth in the hierarchy
# for each of them find all statements (yes, could be made more generic)
def find_initial(module, depth, locations=[]):
    if depth <= 0:
        return locations
    
    processes = module.process_handles()
    for process in processes:
        if (str(process.type()) == "npiInitial"):
            find_all_statements(process, locations)
        
    instances = module.internal_scope_handles()
    for instance in instances:
        find_initial(instance, depth-1, locations)
        
    return locations

top = lang.handle_by_name('top', None)
initial_statements = list(dict.fromkeys(find_initial(top,1)))
print(initial_statements)

[('simple_example/top.v', 9), ('simple_example/top.v', 11), ('simple_example/top.v', 12), ('simple_example/top.v', 13), ('simple_example/top.v', 15), ('simple_example/top.v', 16), ('simple_example/top.v', 18), ('simple_example/top.v', 19), ('simple_example/top.v', 21), ('simple_example/top.v', 22), ('simple_example/top.v', 24), ('simple_example/top.v', 25), ('simple_example/top.v', 27), ('simple_example/top.v', 28), ('simple_example/top.v', 29)]


# Open coverage database

In [6]:
from pynpi import cov

cov_db = cov.open('simv.vdb')
if not cov_db:
    print("Error. Failed to open file: ", vdb_name)

# Traverse database to find line holes

In [13]:
# we might go through the same line multiple times. if we saw it covered once, good enough
def collect_line_holes(block_line_metric, test, block_name, coverage_data):
    childList = block_line_metric.child_handles()
        
    for child in childList:
        if (child.file_name(), child.line_no()) in coverage_data:
            coverage_data[(child.file_name(), child.line_no())] = (child.covered(test) > 0)
        elif child.covered(test):
            coverage_data[(child.file_name(), child.line_no())] = True
    
# this function simply traverses the coverage hierarchy tree
def traverse_cov(inst, test, coverage_data={}):
    block_line_metric = inst.line_metric_handle()
    if (block_line_metric != None):
        collect_line_holes(block_line_metric, test, inst.full_name(), coverage_data)
    
    instList = inst.instance_handles()
    
    for inst in instList:
        traverse_cov(inst, test, coverage_data)
        cov.release_handle(inst)
            
    return coverage_data

def line_holes_from_vdb(cov_db, block_name):
    test = cov_db.test_handles()[0]
    block_inst = cov_db.handle_by_name(block_name)    
    return traverse_cov(block_inst, test)


coverage_data = line_holes_from_vdb(cov_db, 'top')
print(coverage_data)

{('simple_example/top.v', 11): True, ('simple_example/top.v', 13): True, ('simple_example/top.v', 16): False, ('simple_example/top.v', 18): True, ('simple_example/top.v', 19): False, ('simple_example/top.v', 21): True, ('simple_example/top.v', 22): False, ('simple_example/top.v', 24): True, ('simple_example/top.v', 27): True, ('simple_example/top.v', 36): True, ('simple_example/top.v', 37): True, ('simple_example/top.v', 39): True, ('simple_example/top.v', 42): True}


# Filter "initial" lines from holes

In [16]:
for statement in initial_statements:
    if statement in coverage_data:
        coverage_data.pop(statement)
    
print(coverage_data)

{('simple_example/top.v', 36): True, ('simple_example/top.v', 37): True, ('simple_example/top.v', 39): True, ('simple_example/top.v', 42): True}
